In [1]:
import sys, os, itertools
import re, nltk
from nltk import pos_tag
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import sklearn, gensim
from sklearn.decomposition import PCA
from gensim.corpora import Dictionary
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
os.chdir(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/')
from utils import preprocess_for_bow
from models.lda import LDAwrappers
from models.hdp import HDPwrapper
from models.gsdmm import MovieGroupProcessWrapper
from models.lftm import LFTMwrapper
from models.bertopic import BERTopicWrapper
from sklearn.feature_extraction.text import CountVectorizer
from scipy.cluster import hierarchy as sch
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import norm
from numpy import random
from bertopic.representation import MaximalMarginalRelevance, KeyBERTInspired
keybert = KeyBERTInspired()
mmr = MaximalMarginalRelevance(diversity=0.3)
chained_representation = [keybert, mmr]

In [94]:
setopics=["Virus Origin and bioweapon theory", "Virus contraction (who contracts the virus) and transmission (what transmits the virus)", 
                    "Incidence, Mortality and 'Casedemic'", "Prevention (Face Mask, Vitamin D, etc.)", 
                    "Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc)", "Vaccines", "COVID-19 Denialism"]

# dataset 1

In [3]:
data1 = preprocess_for_bow('data.csv', preprocessing=False)
bertopic=BERTopicWrapper(data1['text'], data1['ids'], representation_model=chained_representation)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [5]:
bertopic.find_topics('covid 19 iss such a scam, I swear this is just a conspiracy from Big Pharma', top_n=10)

([63, 100, 648, 76, 532, 363, 56, 70, 12, 53],
 [0.6851774,
  0.6429924,
  0.6380904,
  0.62540233,
  0.6118401,
  0.61149764,
  0.6108924,
  0.60702056,
  0.59038246,
  0.58231115])

In [6]:
bertopic.get_topic_info().iloc[[64,101,649,77,533]]

,Topic,Count,Name,Representation,Representative_Docs
64,63,50,63_covid_covidcoverup_covid19_covid1984,"[covid, covidcoverup, covid19, covid1984, covi...","[COVID IS A HOAX!, Covid is a cover up for the..."
101,100,35,100_coronavirus_patent_coronaviruspandemic_cor...,"[coronavirus, patent, coronaviruspandemic, cor...",[COVID-19 Testing Patent Was Submitted In 2015...
649,648,7,648_covid_pandemic_flu_viruses,"[covid, pandemic, flu, viruses, vaccinations, ...",[There are different strains of covid that's w...
77,76,40,76_covid_treatments_treatment_remedy,"[covid, treatments, treatment, remedy, cure, a...",[A student from Pondicherry University has fou...
533,532,9,532_vaccine_vaccinated_vaccins_covid19,"[vaccine, vaccinated, vaccins, covid19, immuni...",[The vaccine is too fight against so called co...


In [7]:
bertopic.set_topics(setopics, top_n=50, min_sim=0.45)

In [8]:
bertopic.sbert_cluster_set_topics()

## Results from label inputs

In [9]:
print('this print displays the count of docs associated with the topic labels set')
print('')
for i in range(len(setopics)):
    print('topic:', bertopic.topics_set_embeddings[i]['topic_title'], 'yields ', len(bertopic.topics_set_embeddings[i]['topic_idxs']),
            'distributed over', len(bertopic.topics_set_embeddings[i]['topic_labels']),'original bertopics')

this print displays the count of docs associated with the topic labels set

topic: Virus Origin and bioweapon theory yields  187 distributed over 5 original bertopics
topic: Virus contraction (who contracts the virus) and transmission (what transmits the virus) yields  20 distributed over 2 original bertopics
topic: Incidence, Mortality and 'Casedemic' yields  402 distributed over 14 original bertopics
topic: Prevention (Face Mask, Vitamin D, etc.) yields  540 distributed over 2 original bertopics
topic: Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc) yields  79 distributed over 2 original bertopics
topic: Vaccines yields  895 distributed over 38 original bertopics
topic: COVID-19 Denialism yields  13817 distributed over 50 original bertopics


### reclustering corpus on centroids extracted from docs above (SBERT)

In [10]:
np.unique(bertopic.sbert_clusters, return_counts=True)
for i in range(len(setopics)):
    print(bertopic.topics_set_embeddings[i]['topic_title'],'->',np.unique(bertopic.sbert_clusters, return_counts=True)[1][i])

Virus Origin and bioweapon theory -> 7610
Virus contraction (who contracts the virus) and transmission (what transmits the virus) -> 1412
Incidence, Mortality and 'Casedemic' -> 1617
Prevention (Face Mask, Vitamin D, etc.) -> 655
Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc) -> 1978
Vaccines -> 1769
COVID-19 Denialism -> 15150


In [39]:
print('examples clusteres')
np.unique(bertopic.sbert_clusters, return_counts=True)
for i in range(len(setopics)):
    print(bertopic.topics_set_embeddings[i]['topic_title'],'->',data1['text'][bertopic.topics_set_embeddings[i]['topic_idxs'][5]])

examples clusteres
Virus Origin and bioweapon theory -> No lab is capable of engineering it. It would be idiotic to use the Coronavirus for a bioweapon. They'd use a far more deadly pathogen. Coronavirus readily trades genetic material. We've already had 2 zoonotic diseases from the species SARS & MERS. Not man made.
Virus contraction (who contracts the virus) and transmission (what transmits the virus) -> Covid-19 does not exist as a virus, but rather a bacterium that has been exposed to 5G radiation.
Incidence, Mortality and 'Casedemic' -> Fascinating concept. You’re referencing the fact that they aren’t testing the dead only the living? Instead relying on expert opinion. Where did a plane crash death get attributed to covid19? Or a cancer patient dying of cancer get reclassified as a covid19 death?
Prevention (Face Mask, Vitamin D, etc.) -> Vitamin D treatment for COVID-19 patients can reduce deaths by up to 60%.
Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc) -> Hyd

## using examples instead of labels

In [42]:
topic_examples=["Harvard epidemiologist says he is not saying the Corona virus was bioengineered as a bioweapon but a new study says it does have a center segment in its genome encoding a special protein to get it into human cells and it did not get there through natural random mechanisms", 
                "Viruses are dead material expelled from healing cells. Everyone has thousands of viruses inside their body. Viruses do not kill humans, unhealthy immune systems do",
                    "This isn’t new information-6% of the deaths were in people that had no other issues. We know that COVID preys on people with comorbidies-which unfortunately there are a lot of in the US. But that information doesn’t help with your arguement...I realize. But the data makes sense", 
                    "This is a very important study on vitamin D and Covid-19. Its findings are incredibly clear. An 80% reduction in need for ICU and a 60% reduction in deaths, simply by giving a very cheap and very safe therapy - calcifediol, or activated vitamin D.", 
                    "Hydroxychloroquine is not an antiviral it is an antimalarial drug Science isn't 'silly or ideological' it is based on hypothesis testing - the hypothesis that HQ was an effective treatment for #Covid19 was tested and it was found to have no effect As I say sorry to disappoint", 
                    "Worth another listen. 700 babies and children will be injected with the experimental MODERNA vaccine in about 45 days in Rochester, NY. How is this even REMOTELY ethical? How can a BABY ‘consent’ to being a lab rat? @DrMikeMendoza @CountyExecBello", 
                    "HOW.MANY AMERICANS BELIEVE THIS COVID HOAX ?? I DO! I BELIEVE R MILITARY R COPS ICE AGENTS NATIONAL GUARD SHOULD DO WHATS RIGHT ONLY TAKE ORDERS FROM TRUMP (NEVER THESE DEMOCRATE GOVS& MAYORS WHO WERE ELECTED BY FAKE BALLOTS PAID 4 BY SOROS)"]

In [50]:
bertopic.set_topics(topic_examples, top_n=50, min_sim=0.55)
bertopic.sbert_cluster_set_topics()

In [51]:
print('this print displays the count of docs associated with the topic labels set')
print('')
for i in range(len(setopics)):
    print('topic:', i, 'yields ', len(bertopic.topics_set_embeddings[i]['topic_idxs']),
            'distributed over', len(bertopic.topics_set_embeddings[i]['topic_labels']),'original bertopics')

this print displays the count of docs associated with the topic labels set

topic: 0 yields  119 distributed over 5 original bertopics
topic: 1 yields  36 distributed over 3 original bertopics
topic: 2 yields  519 distributed over 18 original bertopics
topic: 3 yields  47 distributed over 2 original bertopics
topic: 4 yields  77 distributed over 3 original bertopics
topic: 5 yields  255 distributed over 11 original bertopics
topic: 6 yields  148 distributed over 8 original bertopics


In [56]:
print('examples clusteres')
np.unique(bertopic.sbert_clusters, return_counts=True)
for i in range(len(setopics)):
    print(setopics[i],'->',data1['text'][bertopic.topics_set_embeddings[i]['topic_idxs'][5]])

examples clusteres
Virus Origin and bioweapon theory -> No lab is capable of engineering it. It would be idiotic to use the Coronavirus for a bioweapon. They'd use a far more deadly pathogen. Coronavirus readily trades genetic material. We've already had 2 zoonotic diseases from the species SARS & MERS. Not man made.
Virus contraction (who contracts the virus) and transmission (what transmits the virus) -> Covid-19 does not exist as a virus, but rather a bacterium that has been exposed to 5G radiation.
Incidence, Mortality and 'Casedemic' -> Fascinating concept. You’re referencing the fact that they aren’t testing the dead only the living? Instead relying on expert opinion. Where did a plane crash death get attributed to covid19? Or a cancer patient dying of cancer get reclassified as a covid19 death?
Prevention (Face Mask, Vitamin D, etc.) -> Vitamin D treatment for COVID-19 patients can reduce deaths by up to 60%.
Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc) -> Hyd

In [73]:
print('reclustering')
np.unique(bertopic.sbert_clusters, return_counts=True)
for i in range(len(setopics)):
    print(setopics[i],'->',np.unique(bertopic.sbert_clusters, return_counts=True)[1][i])

reclustering
Virus Origin and bioweapon theory -> 2319
Virus contraction (who contracts the virus) and transmission (what transmits the virus) -> 7946
Incidence, Mortality and 'Casedemic' -> 2206
Prevention (Face Mask, Vitamin D, etc.) -> 1561
Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc) -> 509
Vaccines -> 1658
COVID-19 Denialism -> 13992


### SBERT threshold exploration

In [61]:
# 105% of distribution with smallest distance to centroids
top_to_remove = 0.9
bottom_to_remove = 1
outliersMinP=bertopic.get_sbert_z_outliers(norm.ppf(bottom_to_remove/2))
outliersMaxP=bertopic.get_sbert_z_outliers(norm.ppf(top_to_remove/2))
for i in range(len(setopics)):
    print(setopics[i])
    n=len(list(set(outliersMinP[i])-set(outliersMaxP[i])))
    print('total', n)
    if n>0:
        print('example:',data1['text'][list(set(outliersMinP[i])-set(outliersMaxP[i]))[random.randint(n)]])
    print('')

Virus Origin and bioweapon theory
total 60
example: Jailed Samsung heir Lee Jae-yong wrote a letter announcing plans to move the company abroad

Virus contraction (who contracts the virus) and transmission (what transmits the virus)
total 3170
example: A religious young soldier mocked for his belief in God miraculously started and drove a jeep that had no engine.

Incidence, Mortality and 'Casedemic'
total 45
example: Canadian Pension Plan payments decreased in 2021

Prevention (Face Mask, Vitamin D, etc.)
total 0

Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc)
total 4
example: Dr. Anthony Fauci wrote a paper that found most 1918-1919 Spanish Influenza pandemic deaths were ""because of bacterial pneumonia from wearing masks.""

Vaccines
total 361
example: Just saw something saying the Covid 19 is going away to fast for them to find a vaccine. You mean to tell me y’all need it around to treat it. I tell you white people are the worst moral and spiritual leaders this wor

In [75]:
# from 50% to 75% of the distribution with smallest distance
top_to_remove = 0.25
bottom_to_remove = 0.5
outliersMinP=bertopic.get_sbert_z_outliers(norm.ppf(bottom_to_remove/2))
outliersMaxP=bertopic.get_sbert_z_outliers(norm.ppf(top_to_remove/2))
for i in range(len(setopics)):
    print(setopics[i])
    n=len(list(set(outliersMinP[i])-set(outliersMaxP[i])))
    print('total', n)
    if n>0:
        print('example:',data1['text'][list(set(outliersMinP[i])-set(outliersMaxP[i]))[random.randint(n)]])
    print('')

Virus Origin and bioweapon theory
total 634
example: COVID-19 contact-tracing apps like Healthy Together and ABTraceTogether are tracking you and also the people in your phone contacts and Facebook friends lists.

Virus contraction (who contracts the virus) and transmission (what transmits the virus)
total 1472
example: James Franco said, ""Coronavirus kills 3,000 people and we lose our shit. Obesity kills ~300,000 people annually and we come out with a new flavor of Doritos.""

Incidence, Mortality and 'Casedemic'
total 1077
example: Thirteen Felician sisters from one convent died from COVID-19.

Prevention (Face Mask, Vitamin D, etc.)
total 1197
example: French study published public health data which appeared to show that people who smoke are 80% less likely to catch COVID-19 than non-smokers of the same age and sex. I also read somewhere that smoking also prevent Chemtrail nanoparticles ready access due to tar.

Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc)
total 

In [60]:
# from 75% to 95% of the distribution with smallest distance - strongest outliers
top_to_remove = 0.05
bottom_to_remove = 0.25
outliersMinP=bertopic.get_sbert_z_outliers(norm.ppf(bottom_to_remove/2))
outliersMaxP=bertopic.get_sbert_z_outliers(norm.ppf(top_to_remove/2))
for i in range(len(setopics)):
    print(setopics[i])
    n=len(list(set(outliersMinP[i])-set(outliersMaxP[i])))
    print('total', n)
    if n>0:
        print('example:',data1['text'][list(set(outliersMinP[i])-set(outliersMaxP[i]))[random.randint(n)]])
    print('')

Virus Origin and bioweapon theory
total 212
example: Viral forward states the timings for the distribution of essential items

Virus contraction (who contracts the virus) and transmission (what transmits the virus)
total 6
example: World Health Organisation (WHO) has taken a ‘U-turn’ and that the asymptomatic COVID-19 patients needn’t be isolated or quarantined as they cannot spread coronavirus.

Incidence, Mortality and 'Casedemic'
total 532
example: Audio clip attributed to Dr Trehan from Medanta Hospitals makes multiple claims including that the COVID-19 situation from “27 March the situation is going to become very serious.”

Prevention (Face Mask, Vitamin D, etc.)
total 76
example: C-vitamin forhindrer coronavirus

Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc)
total 105
example: COVID 19 Kits for home linked to Tata Health

Vaccines
total 0

COVID-19 Denialism
total 1354
example: I believe Covid-19 was a Deep State attack against China's economy. Is this the case

## Removing with respect to distance threshold 

In [76]:
# docs above 80% similarity

threshold = 0.8
idxtokeep=bertopic.get_sbert_norm_outliers(threshold)
for i in range(len(setopics)):
    print(setopics[i])
    n=len(idxtokeep[i])
    print('total', n/np.unique(bertopic.sbert_clusters, return_counts=True)[1][i])
    if n>0:
        print('example:',data1['text'][idxtokeep[i][random.randint(n)]])
    print('')

Virus Origin and bioweapon theory
total 0.8171625700733075
example: @jewlicious @HenMazzig It is an Iranian dish called Salad Shirazi. Israelis have nothing, they colonize everything.

Virus contraction (who contracts the virus) and transmission (what transmits the virus)
total 0.940095645607853
example: Rail freight tanker is stamped with 'COVID-19'.

Incidence, Mortality and 'Casedemic'
total 0.6119673617407072
example: “But from 2013, and in only 7 years of my presidency, we have connected close to 3.5 million households, bringing the number of households connected close to 8 million.”

Prevention (Face Mask, Vitamin D, etc.)
total 0.6194746957078796
example: Coronavirus: ¿Los infectados asintomáticos al final ""no contagian"" a otras personas?

Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc)
total 0.8860510805500982
example: The number visible in the red dot decides the strength of your eyes

Vaccines
total 0.25814234016887816
example: @CawthornforNC Pfizer was not 

In [77]:
# docs above 95% similarity

threshold = 0.95
idxtokeep=bertopic.get_sbert_norm_outliers(threshold)
for i in range(len(setopics)):
    print(setopics[i])
    n=len(idxtokeep[i])
    print('total', n/np.unique(bertopic.sbert_clusters, return_counts=True)[1][i])
    if n>0:
        print('example:',data1['text'][idxtokeep[i][random.randint(n)]])
    print('')

Virus Origin and bioweapon theory
total 0.7744717550668392
example: Incredible! #MarsPerseverance

Virus contraction (who contracts the virus) and transmission (what transmits the virus)
total 0.9217216209413541
example: Video shows Chinese Army provoking India to go to war at Ladakh border

Incidence, Mortality and 'Casedemic'
total 0.5775158658204895
example: CNN anchor decries controversial police unit, Covid-19 deaths in Nigeria

Prevention (Face Mask, Vitamin D, etc.)
total 0.5765534913516976
example: İsrailin oyununu görün.. Yahudi ürünleri almaya devam ederseniz çocuklarınızı kendi elinizle ölümüne sebep olabilirsiniz.

Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc)
total 0.7406679764243614
example: A Centers for Disease Control report released in September shows that masks and face coverings are not effective in preventing the spread of COVID-19

Vaccines
total 0.16103739445114595
example: Purdue University engineers developed a magnetic hair-sized robot that i

# dataset 2

In [4]:
data2 = preprocess_for_bow('claimreviews.csv', preprocessing=False)
bertopic2=BERTopicWrapper(data2['text'], data2['ids'], custom_vectorizer=True, representation_model=chained_representation)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [25]:
bertopic2.set_topics(setopics, top_n=50, min_sim=0.35)

In [26]:
bertopic2.sbert_cluster_set_topics()

In [27]:
print('this print displays the count of docs associated with the topic labels set')
print('')
for i in range(len(setopics)):
    print('topic:', setopics[i], 'yields ', len(bertopic2.topics_set_embeddings[i]['topic_idxs']),
            'distributed over', len(bertopic2.topics_set_embeddings[i]['topic_labels']),'original bertopics')

this print displays the count of docs associated with the topic labels set

topic: Virus Origin and bioweapon theory yields  156 distributed over 10 original bertopics
topic: Virus contraction (who contracts the virus) and transmission (what transmits the virus) yields  98 distributed over 5 original bertopics
topic: Incidence, Mortality and 'Casedemic' yields  803 distributed over 42 original bertopics
topic: Prevention (Face Mask, Vitamin D, etc.) yields  451 distributed over 5 original bertopics
topic: Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc) yields  102 distributed over 4 original bertopics
topic: Vaccines yields  802 distributed over 45 original bertopics
topic: COVID-19 Denialism yields  1129 distributed over 50 original bertopics


In [28]:
np.unique(bertopic2.sbert_clusters, return_counts=True)
for i in range(len(setopics)):
    print(bertopic2.topics_set_embeddings[i]['topic_title'],'->',np.unique(bertopic2.sbert_clusters, return_counts=True)[1][i])

Virus Origin and bioweapon theory -> 253
Virus contraction (who contracts the virus) and transmission (what transmits the virus) -> 317
Incidence, Mortality and 'Casedemic' -> 7937
Prevention (Face Mask, Vitamin D, etc.) -> 7103
Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc) -> 636
Vaccines -> 1539
COVID-19 Denialism -> 210


In [84]:
print('examples clusteres')
np.unique(bertopic2.sbert_clusters, return_counts=True)
for i in range(len(setopics)):
    print(bertopic2.topics_set_embeddings[i]['topic_title'],'->',data2['text'][bertopic2.topics_set_embeddings[i]['topic_idxs'][5]])

examples clusteres
Virus Origin and bioweapon theory -> We’ve been asked to look into claims that the Covid-19 virus hasn’t been isolated, doesn’t fulfil “Koch’s postulates”, and that this means PCR diagnosis tests aren’t working.Honesty in public debate mattersYou can help us take action – and get our regular free emailI’m inIsolating a virusFirstly it is incorrect to say that the virus that causes Covid-19 has not been isolated.Isolating a virus means taking a pure sample of a virus from an infected being so it can be studied. There are numerous reports of the virus being isolated by teams around the world.“SARS-CoV2 has been sampled millions of times over from infected people, including those originally found to be infected in China,” Dr Stephen Griffin, a virologist and Associate Professor at Leeds Institute of Medical Research, told Full Fact.Koch’s postulatesIt’s also incorrect to say that the virus that causes Covid-19 would need to meet Koch’s postulates, primarily because Koch

## using examples instead of labels

In [92]:
topic_examples2=[data2['text'][bertopic2.topics_set_embeddings[0]['topic_idxs'][2]], 
          data2['text'][bertopic2.topics_set_embeddings[-1]['topic_idxs'][1]], 
        data2['text'][bertopic2.topics_set_embeddings[2]['topic_idxs'][1]], 
        data2['text'][bertopic2.topics_set_embeddings[3]['topic_idxs'][0]], 
        data2['text'][bertopic2.topics_set_embeddings[4]['topic_idxs'][0]], 
        data2['text'][bertopic2.topics_set_embeddings[5]['topic_idxs'][0]], 
        data2['text'][bertopic2.topics_set_embeddings[-1]['topic_idxs'][5]]]

In [95]:
bertopic2.set_topics(topic_examples2, top_n=50, min_sim=0.55)
bertopic2.sbert_cluster_set_topics()

In [102]:
print('this print displays the count of docs associated with the topic labels set')
print('')
for i in range(len(setopics)):
    print('topic:', i, 'yields ', len(bertopic2.topics_set_embeddings[i]['topic_idxs']),
            'distributed over', len(bertopic2.topics_set_embeddings[i]['topic_labels']),'original bertopics')

this print displays the count of docs associated with the topic labels set

topic: 0 yields  567 distributed over 15 original bertopics
topic: 1 yields  7555 distributed over 50 original bertopics
topic: 2 yields  7096 distributed over 31 original bertopics
topic: 3 yields  472 distributed over 17 original bertopics
topic: 4 yields  346 distributed over 14 original bertopics
topic: 5 yields  594 distributed over 31 original bertopics
topic: 6 yields  7065 distributed over 21 original bertopics


In [103]:
print('examples clusteres')
np.unique(bertopic2.sbert_clusters, return_counts=True)
for i in range(len(setopics)):
    print(setopics[i],'->',data2['text'][bertopic2.topics_set_embeddings[i]['topic_idxs'][5]])

examples clusteres
Virus Origin and bioweapon theory -> We’ve been asked to look into claims that the Covid-19 virus hasn’t been isolated, doesn’t fulfil “Koch’s postulates”, and that this means PCR diagnosis tests aren’t working.Honesty in public debate mattersYou can help us take action – and get our regular free emailI’m inIsolating a virusFirstly it is incorrect to say that the virus that causes Covid-19 has not been isolated.Isolating a virus means taking a pure sample of a virus from an infected being so it can be studied. There are numerous reports of the virus being isolated by teams around the world.“SARS-CoV2 has been sampled millions of times over from infected people, including those originally found to be infected in China,” Dr Stephen Griffin, a virologist and Associate Professor at Leeds Institute of Medical Research, told Full Fact.Koch’s postulatesIt’s also incorrect to say that the virus that causes Covid-19 would need to meet Koch’s postulates, primarily because Koch

In [104]:
print('reclustering')
np.unique(bertopic2.sbert_clusters, return_counts=True)
for i in range(len(setopics)):
    print(setopics[i],'->',np.unique(bertopic2.sbert_clusters, return_counts=True)[1][i])

reclustering
Virus Origin and bioweapon theory -> 6824
Virus contraction (who contracts the virus) and transmission (what transmits the virus) -> 3819
Incidence, Mortality and 'Casedemic' -> 728
Prevention (Face Mask, Vitamin D, etc.) -> 150
Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc) -> 3194
Vaccines -> 952
COVID-19 Denialism -> 2328


## z-test: exploration of outliers

In [105]:
# 25% of distribution with smallest distance to centroids
top_to_remove = 0.75
bottom_to_remove = 1
outliersMinP=bertopic2.get_sbert_z_outliers(norm.ppf(bottom_to_remove/2))
outliersMaxP=bertopic2.get_sbert_z_outliers(norm.ppf(top_to_remove/2))
for i in range(len(setopics)):
    print(setopics[i])
    n=len(list(set(outliersMinP[i])-set(outliersMaxP[i])))
    print('total', n)
    if n>0:
        print('example:',data2['text'][list(set(outliersMinP[i])-set(outliersMaxP[i]))[random.randint(n)]])
    print('')

Virus Origin and bioweapon theory
total 3217
example: Did LeBron James and Anthony Davis, the Los Angeles Lakers' leading scorers this season, wear lace collars to honor the life of Supreme Court Justice Ruth Bader Ginsburg after her death? No, that's not true: This is a satirical claim published by a known satire website. Neither James, Davis, nor any other NBA players have worn lace collars in the couple games that have happened since her death.The claim originated from an article published by ""The Babylon Bee"" on September 22, 2020, titled ""NBA Players Wear Special Lace Collars To Honor Ruth Bader Ginsburg"" (archived here) which opened:LOS ANGELES, CA--NBA players are honoring the life of Ruth Bader Ginsburg this week by wearing pretty lace collars just like Notorious RBG used to wear. In a touching show of respect for the late Justice Ginsburg, and in solidarity with her progressive cause, Lebron James and the LA Lakers took to the court yesterday wearing a stunning variety of 

In [106]:
# from 50% to 75% of the distribution with smallest distance
top_to_remove = 0.25
bottom_to_remove = 0.5
outliersMinP=bertopic2.get_sbert_z_outliers(norm.ppf(bottom_to_remove/2))
outliersMaxP=bertopic2.get_sbert_z_outliers(norm.ppf(top_to_remove/2))
for i in range(len(setopics)):
    print(setopics[i])
    n=len(list(set(outliersMinP[i])-set(outliersMaxP[i])))
    print('total', n)
    if n>0:
        print('example:',data2['text'][list(set(outliersMinP[i])-set(outliersMaxP[i]))[random.randint(n)]])
    print('')

Virus Origin and bioweapon theory
total 741
example: FACT CHECK: Viral Image Claims To Show Nancy Pelosi Violating COVID-19 Safety Guidelines For Social GatheringsAn image shared on Facebook claims to show House Speaker Nancy Pelosi violating coronavirus safety guidelines by attending a party and not wearing a mask.Verdict: FalseThe image was taken in January 2020, over a week before the U.S. confirmed its first COVID-19 case.Fact Check:Under phase two coronavirus restrictions put in place by D.C. Mayor Muriel Bowser, indoor gatherings are currently limited to 10 people, and restaurants are limited to operating at 25 percent capacity indoors. Masks must be worn inside public establishments except while eating or drinking, according to the guidelines.A post on Facebook shares an image of Pelosi gathered around several people. No one appears to be wearing a mask or practicing social distancing. The photo’s caption reads, “Ladies and Gentlemen here’s your speaker of the house enjoying a p

In [107]:
# from 75% to 95% of the distribution with smallest distance - strongest outliers
top_to_remove = 0.05
bottom_to_remove = 0.25
outliersMinP=bertopic2.get_sbert_z_outliers(norm.ppf(bottom_to_remove/2))
outliersMaxP=bertopic2.get_sbert_z_outliers(norm.ppf(top_to_remove/2))
for i in range(len(setopics)):
    print(setopics[i])
    n=len(list(set(outliersMinP[i])-set(outliersMaxP[i])))
    print('total', n)
    if n>0:
        print('example:',data2['text'][list(set(outliersMinP[i])-set(outliersMaxP[i]))[random.randint(n)]])
    print('')

Virus Origin and bioweapon theory
total 514
example: The spread of novel coronavirus, which was first declared as a pandemic by the World Health Organisation (WHO) on 11 March has since then become a target of misinformation and conspiracy theories alike.In one such instance, several users on social media are claiming that the 16th century French astrologer Nostradamus predicted the virus in his book ‘Les Propheties’ (The Prophecies). However, after proper checks and verification, we have found that there is no truth to these claims.THE CLAIMAccording to the claim being circulated on the internet, French philosopher Nostradamus in his book, ‘Les Propheties’ (The Prophecies) wrote: “There will be a twin year (2020) from which will arise a queen (corona) who will come from the east (China) and will spread a plague (virus)in the darkness of night, on a country with 7 hills (Italy) and will transform the twilight of men into dust (death), to destroy, and ruin the world. (sic)”WHAT WE FOUND

## exploration via distance measure

In [108]:
# docs above 80% similarity

threshold = 0.8
idxtokeep=bertopic2.get_sbert_norm_outliers(threshold)
for i in range(len(setopics)):
    print(setopics[i])
    n=len(idxtokeep[i])
    print('total', n/np.unique(bertopic2.sbert_clusters, return_counts=True)[1][i])
    if n>0:
        print('example:',data2['text'][idxtokeep[i][random.randint(n)]])
    print('')

Virus Origin and bioweapon theory
total 0.9673212192262602
example: The number of mail collection boxes declined by more than 12,000 nationally from 2011 to 2016, according to an August 2016 report, largely because they were underused.It is difficult to prove that the removal of these mailboxes during the Obama administration was politically motivated, or at all connected to elections.Amidst mounting concerns over the U.S. Postal Service’s ability to handle an influx of mail-in ballots for the November 2020 election due to COVID-19 quarantines, a spate of mailbox removals across the country in the summer of 2020 led to accusations from Democrats that U.S. President Donald Trump’s Republican administration was hindering the voting process. In response, numerous conservative media outlets argued in August 2020 that thousands of mailboxes were also removed during U.S. President Barack Obama's administration — mainly from 2011 to 2016.Snopes readers asked us to verify whether mailboxes had

In [109]:
# docs above 95% similarity

threshold = 0.95
idxtokeep=bertopic2.get_sbert_norm_outliers(threshold)
for i in range(len(setopics)):
    print(setopics[i])
    n=len(idxtokeep[i])
    print('total', n/np.unique(bertopic2.sbert_clusters, return_counts=True)[1][i])
    if n>0:
        print('example:',data2['text'][idxtokeep[i][random.randint(n)]])
    print('')

Virus Origin and bioweapon theory
total 0.9572098475967175
example: Pennsylvania’s governor did not convert to IslamFacebook posts shared thousands of times link to an article claiming that Pennsylvania Governor Tom Wolf converted to Islam. This is false; his office says he is Christian, and the photos in the article are from a Muslim religious event he hosted in an official capacity.“It has become evident through the photographs that Wolf has in fact already converted. According to Islam it is not permissible for a non-Muslim to pray either in the ranks or on his own,” according to the article, which has been shared here and here on Facebook, here on Twitter and here on Reddit.“He needs to come out and explain to the voters in his state as to why he converted,” a user commented on one of the Facebook posts, while others suggested he should be removed from office.The article claiming that Wolf converted to Islam shows pictures of him joining Muslims in prayer, but they were shot in Aug